In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)      # Mostrar todas las columnas
pd.set_option('display.width', 1000)            # Ancho de línea amplio
pd.set_option('display.max_colwidth', 30)       # Limitar ancho de cada columna
pd.set_option('display.precision', 2)           # Precisión decimal
# Cargar datos
banco_df = pd.read_csv('bank-additional.csv')
cliente_df = pd.read_excel('customer-details.xlsx', sheet_name='2012')

print(f"Datos cargados:")
print(f"banco_df: {banco_df.shape}")
print(f"cliente_df: {cliente_df.shape}")
print(banco_df.head())
print(cliente_df.head())



Datos cargados:
banco_df: (43000, 24)
cliente_df: (20115, 7)
   Unnamed: 0   age        job  marital    education  default  housing  loan    contact  duration  campaign  pdays  previous     poutcome  emp.var.rate cons.price.idx cons.conf.idx euribor3m nr.employed   y                date  latitude  longitude                            id_
0           0   NaN  housemaid  MARRIED     basic.4y      0.0      0.0   0.0  telephone       261         1    999         0  NONEXISTENT           1.1         93,994         -36,4     4,857        5191  no       2-agosto-2019     41.49     -71.23  089b39d8-e4d0-461b-87d4-81...
1           1  57.0   services  MARRIED  high.school      NaN      0.0   0.0  telephone       149         1    999         0  NONEXISTENT           1.1         93,994         -36,4       NaN        5191  no  14-septiembre-2016     34.60     -83.92  e9d37224-cb6f-4942-98d7-46...
2           2  37.0   services  MARRIED  high.school      0.0      1.0   0.0  telephone       226     

In [5]:
# crear copias
banco_limpio = banco_df.copy()
cliente_limpio = cliente_df.copy()
print(f"Copias creadas:")
print(f"  banco_limpio: {banco_limpio.shape}")
print(f"  cliente_limpio: {cliente_limpio.shape}")


Copias creadas:
  banco_limpio: (43000, 24)
  cliente_limpio: (20115, 7)


In [6]:
#empezamos a limpiar y transformar los datos
# vemos los duplicados
duplicados_banco = banco_df.duplicated().sum()
duplicados_cliente = cliente_df.duplicated().sum()

print(f"Duplicados completos en banco_df: {duplicados_banco}")
print(f"Duplicados completos en cliente_df: {duplicados_cliente}")

Duplicados completos en banco_df: 0
Duplicados completos en cliente_df: 0


In [7]:
#vemos valores faltantes (NaN)
print("valores faltantes en banco_df:")
faltantes_banco = banco_df.isnull().sum()
print(faltantes_banco[faltantes_banco > 0])


valores faltantes en banco_df:
age               5120
job                345
marital             85
education         1807
default           8981
housing           1026
loan              1026
cons.price.idx     471
euribor3m         9256
date               248
dtype: int64


In [8]:
#vemos faltantes en cliente_df
print("valores faltantes en cliente_df:")
faltantes_cliente = cliente_df.isnull().sum()
print(faltantes_cliente[faltantes_cliente > 0])


valores faltantes en cliente_df:
Series([], dtype: int64)


In [9]:
# Porcentaje de valores faltantes
print("\nPorcentaje de valores faltantes en banco_df:")
for col in banco_df.columns:
    porcentaje = (banco_df[col].isnull().sum() / len(banco_df)) * 100
    if porcentaje > 0:
        print(f"{col}: {porcentaje:.1f}%")




Porcentaje de valores faltantes en banco_df:
age: 11.9%
job: 0.8%
marital: 0.2%
education: 4.2%
default: 20.9%
housing: 2.4%
loan: 2.4%
cons.price.idx: 1.1%
euribor3m: 21.5%
date: 0.6%


In [10]:
# problema con los datos
# Problema 1: Columna innecesaria 'Unnamed: 0'
print("Columnas que empiezan con 'Unnamed':")
for col in banco_df.columns:
    if 'Unnamed' in col:
        print(f"- {col}")
print("Ejemplos de valores en 'Unnamed: 0':")
for col in cliente_df.columns:
    if 'Unnamed' in col:
        print(f"- {col}:")
print("Ejemplos de valores en 'Unnamed: 0':")


Columnas que empiezan con 'Unnamed':
- Unnamed: 0
Ejemplos de valores en 'Unnamed: 0':
- Unnamed: 0:
Ejemplos de valores en 'Unnamed: 0':


In [11]:
# Usar la ruta simple - están en el directorio principal
datos_originales = pd.read_csv('bank-additional.csv')
print(datos_originales.dtypes)
print(datos_originales['cons.price.idx'].head())

Unnamed: 0          int64
age               float64
job                object
marital            object
education          object
default           float64
housing           float64
loan              float64
contact            object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx     object
cons.conf.idx      object
euribor3m          object
nr.employed        object
y                  object
date               object
latitude          float64
longitude         float64
id_                object
dtype: object
0    93,994
1    93,994
2    93,994
3    93,994
4    93,994
Name: cons.price.idx, dtype: object


In [12]:
# problemas fechas en texto
print("\nColumnas de fecha:")
for col in banco_df.columns:
    if 'date' in col.lower():
        print(f"- {col}: tipo {banco_df[col].dtype}")
        print(f"  Ejemplos: {banco_df[col].head(3).tolist()}")



Columnas de fecha:
- date: tipo object
  Ejemplos: ['2-agosto-2019', '14-septiembre-2016', '15-febrero-2019']


In [13]:
# PASO 1: Eliminar columnas innecesarias
if 'Unnamed: 0' in banco_limpio.columns:
    banco_limpio = banco_limpio.drop('Unnamed: 0', axis=1)
    print("PASO 1: Eliminada 'Unnamed: 0' de banco")

if 'Unnamed: 0' in cliente_limpio.columns:
    cliente_limpio = cliente_limpio.drop('Unnamed: 0', axis=1)
    print("PASO 1: Eliminada 'Unnamed: 0' de cliente")


PASO 1: Eliminada 'Unnamed: 0' de banco
PASO 1: Eliminada 'Unnamed: 0' de cliente


In [14]:
# PASO 2: Eliminar duplicados
banco_limpio = banco_limpio.drop_duplicates()

In [15]:
# PASO 3: Formato de columnas numéricas
# Columnas que deberían ser numéricas pero son object
columnas_numericas = ['cons.price.idx', 'cons.conf.idx', 'euribor3m', 'emp.var.rate', 'nr.employed']

for col in columnas_numericas:
    if col in banco_limpio.columns:
        print(f"\nProcesando {col}:")
        tipo_antes = banco_limpio[col].dtype
        print(f"  Tipo antes: {tipo_antes}")
        
        if tipo_antes == 'object':
            # Mostrar algunos valores antes de convertir
            valores_ejemplo = banco_limpio[col].dropna().head(3).tolist()
            print(f"  Valores ejemplo: {valores_ejemplo}")
            
            # Verificar si tiene comas (formato europeo)
            tiene_comas = banco_limpio[col].dropna().astype(str).str.contains(',').any()
            print(f"  Tiene formato europeo (comas): {tiene_comas}")
            
            if tiene_comas:
                # Convertir comas a puntos
                banco_limpio[col] = banco_limpio[col].astype(str).str.replace(',', '.')
                print("  Comas convertidas a puntos")
            
            # Convertir a numérico
            nulos_antes = banco_limpio[col].isnull().sum()
            banco_limpio[col] = pd.to_numeric(banco_limpio[col], errors='coerce')
            nulos_despues = banco_limpio[col].isnull().sum()
            
            tipo_despues = banco_limpio[col].dtype
            nulos_creados = nulos_despues - nulos_antes
            
            print(f"  Resultado: {tipo_antes} → {tipo_despues}")
            print(f"  NaN creados en conversión: {nulos_creados}")
            
            # Mostrar algunos valores después de convertir
            if banco_limpio[col].notna().sum() > 0:
                valores_convertidos = banco_limpio[col].dropna().head(3).tolist()
                print(f"  Valores convertidos: {valores_convertidos}")
            
            if nulos_creados < len(banco_limpio) * 0.5:  # Si no se crearon muchos NaN
                print(f"  ✅ Conversión exitosa")
            else:
                print(f"  ⚠️ Muchos NaN creados - revisar datos")
        else:
            print(f"  ✅ Ya era numérico: {tipo_antes}")




Procesando cons.price.idx:
  Tipo antes: object
  Valores ejemplo: ['93,994', '93,994', '93,994']
  Tiene formato europeo (comas): True
  Comas convertidas a puntos
  Resultado: object → float64
  NaN creados en conversión: 471
  Valores convertidos: [93.994, 93.994, 93.994]
  ✅ Conversión exitosa

Procesando cons.conf.idx:
  Tipo antes: object
  Valores ejemplo: ['-36,4', '-36,4', '-36,4']
  Tiene formato europeo (comas): True
  Comas convertidas a puntos
  Resultado: object → float64
  NaN creados en conversión: 0
  Valores convertidos: [-36.4, -36.4, -36.4]
  ✅ Conversión exitosa

Procesando euribor3m:
  Tipo antes: object
  Valores ejemplo: ['4,857', '4,857', '4,857']
  Tiene formato europeo (comas): True
  Comas convertidas a puntos
  Resultado: object → float64
  NaN creados en conversión: 9256
  Valores convertidos: [4.857, 4.857, 4.857]
  ✅ Conversión exitosa

Procesando emp.var.rate:
  Tipo antes: float64
  ✅ Ya era numérico: float64

Procesando nr.employed:
  Tipo antes: obj

In [16]:
print(f"\nPASO 3 completado - Verificar resultado:")
for col in columnas_numericas:
    if col in banco_limpio.columns:
        nulos = banco_limpio[col].isnull().sum()
        total = len(banco_limpio)
        porcentaje = (nulos / total) * 100
        print(f"  {col}: {banco_limpio[col].dtype}, {nulos} NaN ({porcentaje:.1f}%)")



PASO 3 completado - Verificar resultado:
  cons.price.idx: float64, 471 NaN (1.1%)
  cons.conf.idx: float64, 0 NaN (0.0%)
  euribor3m: float64, 9256 NaN (21.5%)
  emp.var.rate: float64, 0 NaN (0.0%)
  nr.employed: float64, 0 NaN (0.0%)


In [18]:
# PASO 3: Pasar age de float a integer

# Verificar estado actual
if 'age' in banco_limpio.columns:
    print(f"Tipo actual: {banco_limpio['age'].dtype}")
    print(f"Valores ejemplo: {banco_limpio['age'].head(5).tolist()}")
    print(f"Valores faltantes: {banco_limpio['age'].isnull().sum()}")
    
    # Verificar si hay valores decimales
    edad_con_decimales = banco_limpio['age'].dropna() % 1 != 0
    if edad_con_decimales.any():
        print(f"ADVERTENCIA: {edad_con_decimales.sum()} valores tienen decimales")
        print("Valores con decimales:", banco_limpio['age'][edad_con_decimales].head().tolist())
        print("Se redondearán al entero más cercano")
    
    # Convertir a integer
    if banco_limpio['age'].isnull().sum() == 0:
        # Si no hay NaN, conversión directa
        banco_limpio['age'] = banco_limpio['age'].astype(int)
        print("Conversión directa (sin NaN)")
    else:
        # Si hay NaN, usar Int64 (permite NaN)
        banco_limpio['age'] = banco_limpio['age'].round().astype('Int64')
        print("Conversión con Int64 (mantiene NaN)")
    
    # Verificar resultado
    print(f"Tipo después: {banco_limpio['age'].dtype}")
    print(f"Valores ejemplo después: {banco_limpio['age'].head(5).tolist()}")
    print(f"Rango de edad: {banco_limpio['age'].min()} a {banco_limpio['age'].max()}")
    
    # Verificar que no se perdieron datos
    print(f"Total valores: {len(banco_limpio['age'])}")
    print(f"Valores no nulos: {banco_limpio['age'].notna().sum()}")
    
    print("Conversión completada")
else:
    print("No se encontró la columna 'age'")

Tipo actual: Int64
Valores ejemplo: [<NA>, 57, 37, 40, 56]
Valores faltantes: 5120
Conversión con Int64 (mantiene NaN)
Tipo después: Int64
Valores ejemplo después: [<NA>, 57, 37, 40, 56]
Rango de edad: 17 a 98
Total valores: 43000
Valores no nulos: 37880
Conversión completada


In [19]:
# PASO 4: Convertir fechas españolas a datetime


if 'date' in banco_limpio.columns:
    print(f"Tipo actual de date: {banco_limpio['date'].dtype}")
    print(f"Fechas ejemplo: {banco_limpio['date'].head(3).tolist()}")
    
    if banco_limpio['date'].dtype == 'object':
        print("Traduciendo meses del español al inglés...")
        
        # Traducir todos los meses
        banco_limpio['date'] = banco_limpio['date'].str.replace('enero', 'January')
        banco_limpio['date'] = banco_limpio['date'].str.replace('febrero', 'February')
        banco_limpio['date'] = banco_limpio['date'].str.replace('marzo', 'March')
        banco_limpio['date'] = banco_limpio['date'].str.replace('abril', 'April')
        banco_limpio['date'] = banco_limpio['date'].str.replace('mayo', 'May')
        banco_limpio['date'] = banco_limpio['date'].str.replace('junio', 'June')
        banco_limpio['date'] = banco_limpio['date'].str.replace('julio', 'July')
        banco_limpio['date'] = banco_limpio['date'].str.replace('agosto', 'August')
        banco_limpio['date'] = banco_limpio['date'].str.replace('septiembre', 'September')
        banco_limpio['date'] = banco_limpio['date'].str.replace('octubre', 'October')
        banco_limpio['date'] = banco_limpio['date'].str.replace('noviembre', 'November')
        banco_limpio['date'] = banco_limpio['date'].str.replace('diciembre', 'December')
        
        print(f"Fechas traducidas: {banco_limpio['date'].head(3).tolist()}")
        
        # Convertir a datetime
        print("Convirtiendo a datetime...")
        banco_limpio['date'] = pd.to_datetime(banco_limpio['date'], errors='coerce')
        
        # Verificar resultado
        fechas_validas = banco_limpio['date'].notna().sum()
        total_fechas = len(banco_limpio['date'])
        porcentaje_exito = (fechas_validas / total_fechas) * 100
        
        print(f"Resultado:")
        print(f"  Fechas válidas: {fechas_validas:,}/{total_fechas:,}")
        print(f"  Porcentaje éxito: {porcentaje_exito:.1f}%")
        print(f"  Tipo final: {banco_limpio['date'].dtype}")
        
        if porcentaje_exito > 95:
            print("  Conversión exitosa")
            
            # Crear variables derivadas de fecha
            print("Creando variables de fecha...")
            banco_limpio['año'] = banco_limpio['date'].dt.year
            banco_limpio['mes'] = banco_limpio['date'].dt.month
            banco_limpio['trimestre'] = banco_limpio['date'].dt.quarter
            banco_limpio['dia_semana'] = banco_limpio['date'].dt.dayofweek
            
            print("Variables creadas: año, mes, trimestre, dia_semana")
            
            # Mostrar distribución por años
            print("Distribución por años:")
            distribucion = banco_limpio['año'].value_counts().sort_index()
            for año, cantidad in distribucion.items():
                print(f"  {año}: {cantidad:,} registros")
                
        else:
            print("  Hay problemas en la conversión")
            print(f"  Fechas problemáticas: {total_fechas - fechas_validas}")
            
    else:
        print(f"La columna ya es {banco_limpio['date'].dtype}")
        print(f"Rango: {banco_limpio['date'].min()} a {banco_limpio['date'].max()}")

# También convertir fecha en cliente
if 'Dt_Customer' in cliente_limpio.columns:
    cliente_limpio['Dt_Customer'] = pd.to_datetime(cliente_limpio['Dt_Customer'], errors='coerce')
    print("Dt_Customer en cliente_limpio convertida")

print("PASO 4 completado")

Tipo actual de date: object
Fechas ejemplo: ['2-agosto-2019', '14-septiembre-2016', '15-febrero-2019']
Traduciendo meses del español al inglés...
Fechas traducidas: ['2-August-2019', '14-September-2016', '15-February-2019']
Convirtiendo a datetime...
Resultado:
  Fechas válidas: 42,752/43,000
  Porcentaje éxito: 99.4%
  Tipo final: datetime64[ns]
  Conversión exitosa
Creando variables de fecha...
Variables creadas: año, mes, trimestre, dia_semana
Distribución por años:
  2015.0: 8,544 registros
  2016.0: 8,533 registros
  2017.0: 8,562 registros
  2018.0: 8,549 registros
  2019.0: 8,564 registros
Dt_Customer en cliente_limpio convertida
PASO 4 completado


In [20]:
#paso 5 Rellenar valores faltantes

# Ver cuántos valores faltantes hay
total_faltantes = banco_limpio.isnull().sum().sum()
print(f"Total valores faltantes: {total_faltantes}")

# Mostrar por columna
print("Por columna:")
for col in banco_limpio.columns:
    faltantes = banco_limpio[col].isnull().sum()
    if faltantes > 0:
        print(f"  {col}: {faltantes}")

# Rellenar cada tipo de columna
print("\nRellenando...")

# Para edad: usar la mediana
if banco_limpio['age'].isnull().sum() > 0:
    banco_limpio['age'] = banco_limpio['age'].fillna(banco_limpio['age'].median())
    print("Age: rellenado con mediana")

# Para default, housing, loan: usar 0
for col in ['default', 'housing', 'loan']:
    if col in banco_limpio.columns:
        banco_limpio[col] = banco_limpio[col].fillna(0)
        print(f"{col}: rellenado con 0")

# Para texto: usar la moda (más frecuente)
for col in ['job', 'marital', 'education']:
    if col in banco_limpio.columns and banco_limpio[col].isnull().sum() > 0:
        moda = banco_limpio[col].mode()[0]
        banco_limpio[col] = banco_limpio[col].fillna(moda)
        print(f"{col}: rellenado con '{moda}'")

# Para números: usar la mediana
for col in ['cons.price.idx', 'euribor3m']:
    if col in banco_limpio.columns and banco_limpio[col].isnull().sum() > 0:
        mediana = banco_limpio[col].median()
        banco_limpio[col] = banco_limpio[col].fillna(mediana)
        print(f"{col}: rellenado con {mediana}")

# Verificar resultado
print("\nResultado:")
faltantes_final = banco_limpio.isnull().sum().sum()
print(f"Valores faltantes después: {faltantes_final}")

if faltantes_final == 0:
    print("Dataset completamente limpio")
else:
    print("Todavía hay algunos valores faltantes")

print(f"Filas: {len(banco_limpio)}")
print(f"Columnas: {len(banco_limpio.columns)}")
print("PASO 5 terminado")

Total valores faltantes: 29357
Por columna:
  age: 5120
  job: 345
  marital: 85
  education: 1807
  default: 8981
  housing: 1026
  loan: 1026
  cons.price.idx: 471
  euribor3m: 9256
  date: 248
  año: 248
  mes: 248
  trimestre: 248
  dia_semana: 248

Rellenando...
Age: rellenado con mediana
default: rellenado con 0
housing: rellenado con 0
loan: rellenado con 0
job: rellenado con 'admin.'
marital: rellenado con 'MARRIED'
education: rellenado con 'university.degree'
cons.price.idx: rellenado con 93.749
euribor3m: rellenado con 4.857

Resultado:
Valores faltantes después: 1240
Todavía hay algunos valores faltantes
Filas: 43000
Columnas: 27
PASO 5 terminado


In [23]:
# Rellenar fechas faltantes en lugar de eliminar filas

# Ver el problema
print(f"Filas con fecha faltante: {banco_limpio['date'].isnull().sum()}")
print(f"Total filas: {len(banco_limpio)}")

# Calcular fecha promedio de las fechas existentes
fechas_validas = banco_limpio['date'].dropna()
fecha_promedio = fechas_validas.mean()

print(f"Fecha promedio calculada: {fecha_promedio}")
print(f"Fecha promedio (formato): {fecha_promedio.strftime('%Y-%m-%d')}")

# Rellenar fechas faltantes con la fecha promedio
banco_limpio['date'] = banco_limpio['date'].fillna(fecha_promedio)

print(f"Fechas faltantes después: {banco_limpio['date'].isnull().sum()}")

# Recalcular las variables de fecha
print("Recalculando variables de fecha...")
banco_limpio['año'] = banco_limpio['date'].dt.year
banco_limpio['mes'] = banco_limpio['date'].dt.month
banco_limpio['trimestre'] = banco_limpio['date'].dt.quarter
banco_limpio['dia_semana'] = banco_limpio['date'].dt.dayofweek

# Verificar resultado final
total_faltantes = banco_limpio.isnull().sum().sum()
print(f"\nValores faltantes totales: {total_faltantes}")

if total_faltantes == 0:
    print("Dataset completamente limpio!")
    print("Se conservaron todas las filas")
else:
    print("Todavía hay algunos faltantes:")
    for col in banco_limpio.columns:
        faltantes = banco_limpio[col].isnull().sum()
        if faltantes > 0:
            print(f"  {col}: {faltantes}")

print(f"\nResumen final:")
print(f"Filas: {len(banco_limpio):,}")
print(f"Columnas: {len(banco_limpio.columns)}")
print(f"Rango de fechas: {banco_limpio['date'].min()} a {banco_limpio['date'].max()}")

# Ver distribución por años después del rellenado
print(f"\nDistribución por años:")
distribucion = banco_limpio['año'].value_counts().sort_index()
for año, cantidad in distribucion.items():
    print(f"  {año}: {cantidad:,} registros")

print(f"\nDatos listos para análisis exploratorio!")

Filas con fecha faltante: 0
Total filas: 43000
Fecha promedio calculada: 2017-07-01 19:55:11.676646656
Fecha promedio (formato): 2017-07-01
Fechas faltantes después: 0
Recalculando variables de fecha...

Valores faltantes totales: 0
Dataset completamente limpio!
Se conservaron todas las filas

Resumen final:
Filas: 43,000
Columnas: 27
Rango de fechas: 2015-01-01 00:00:00 a 2019-12-31 00:00:00

Distribución por años:
  2015: 8,544 registros
  2016: 8,533 registros
  2017: 8,810 registros
  2018: 8,549 registros
  2019: 8,564 registros

Datos listos para análisis exploratorio!


In [24]:

# Ahora guardar los archivos
print("Guardando datos limpios...")
banco_limpio.to_csv('banco_limpio.csv', index=False)
cliente_limpio.to_csv('cliente_limpio.csv', index=False)
print("Archivos guardados exitosamente!")

# Verificar que se guardaron
print(f"banco_limpio.csv: {banco_limpio.shape}")
print(f"cliente_limpio.csv: {cliente_limpio.shape}")

Guardando datos limpios...
Archivos guardados exitosamente!
banco_limpio.csv: (43000, 27)
cliente_limpio.csv: (20115, 6)


In [25]:
# ANÁLISIS DESCRIPTIVO

# Lo básico que necesitas
print("Tamaño del dataset:", banco_limpio.shape)
print("\nTipos de datos:")
print(banco_limpio.dtypes)

print("\nEstadísticas básicas:")
print(banco_limpio.describe())

print("\nValores únicos de la variable objetivo:")
print(banco_limpio['y'].value_counts())

Tamaño del dataset: (43000, 27)

Tipos de datos:
age                        Int64
job                       object
marital                   object
education                 object
default                  float64
housing                  float64
loan                     float64
contact                   object
duration                   int64
campaign                   int64
pdays                      int64
previous                   int64
poutcome                  object
emp.var.rate             float64
cons.price.idx           float64
cons.conf.idx            float64
euribor3m                float64
nr.employed              float64
y                         object
date              datetime64[ns]
latitude                 float64
longitude                float64
id_                       object
año                        int32
mes                        int32
trimestre                  int32
dia_semana                 int32
dtype: object

Estadísticas básicas:
           age   defaul